# Cumulative Strategy - 5 Class Grouped

In [1]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler as MMS
#torch
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.nn import BCEWithLogitsLoss,BCELoss

#avalanche
from avalanche.models import SimpleMLP,MlpVAE,SlimResNet18,IcarlNet #model
from avalanche.training.supervised import Naive,Cumulative,Replay,LwF,ICaRL #cl strat
from avalanche.benchmarks.generators import nc_benchmark,ni_benchmark #custom benchmark
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics #metrics
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, CSVLogger #loggers
from avalanche.training.plugins import EvaluationPlugin


/Users/jesherjoshua/Library/Python/3.8/lib/python/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jesherjoshua/Library/Python/3.8/lib/python/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: /Users/jesherjoshua/Library/Python/3.8/lib/python/site-packages/torchvision/image.so
  Expected in: /Users/jesherjoshua/Library/Python/3.8/lib/python/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
df_train=pd.read_csv('../data/fiveclass_train.csv')
df_train=df_train.drop(['Fwd Avg Bytes/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd PSH Flags','Bwd URG Flags','Bwd Avg Bulk Rate','Fwd Avg Packets/Bulk','Bwd Avg Packets/Bulk'],axis=1)
df_test=pd.read_csv('../data/fiveclass_test.csv')
df_test=df_test.drop(['Fwd Avg Bytes/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd PSH Flags','Bwd URG Flags','Bwd Avg Bulk Rate','Fwd Avg Packets/Bulk','Bwd Avg Packets/Bulk'],axis=1)


In [3]:
df_train.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,fiveLabels
0,80,6018089.0,5,3,177,994,159,0,35.400000,69.143329,...,275923.0000,0.000,275923,275923,5742166.0,0.000000,5742166,5742166,0,0
1,443,323049.0,8,6,531,3208,194,0,66.375000,85.880211,...,0.0000,0.000,0,0,0.0,0.000000,0,0,0,0
2,80,39270118.0,9,10,898,3944,431,0,99.777778,187.795619,...,123253.6667,149943.928,296394,36667,10000000.0,300.561142,10000000,9999980,0,0
3,4848,43.0,1,1,2,6,2,2,2.000000,0.000000,...,0.0000,0.000,0,0,0.0,0.000000,0,0,10,3
4,80,5754816.0,3,1,12,0,6,0,4.000000,3.464102,...,0.0000,0.000,0,0,0.0,0.000000,0,0,0,0


# Load Data

In [4]:
x_train=df_train.drop(['Label','fiveLabels'],axis=1)
y_train=df_train.fiveLabels
x_test=df_test.drop(['Label','fiveLabels'],axis=1)
y_test=df_test.fiveLabels
x_train.shape,x_test.shape,y_train.shape
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values

# Create Benchmarks

In [5]:

mms=MMS(feature_range=(0,1))
x_train=mms.fit_transform(x_train)
x_test=mms.fit_transform(x_test)
x_train=torch.tensor(x_train,dtype=torch.float32)
y_train=torch.tensor(y_train)
x_test=torch.tensor(x_test,dtype=torch.float32)
y_test=torch.tensor(y_test)

In [6]:
type(x_train)

torch.Tensor

In [7]:
train_dataset=TensorDataset(x_train,y_train)
test_dataset=TensorDataset(x_test,y_test)
NC_benchmark = nc_benchmark(train_dataset=train_dataset,test_dataset = test_dataset,n_experiences=6,task_labels=False,shuffle=False,fixed_class_order=[0,1,3,2,5,4])
#NI_benchmark=ni_benchmark(train_dataset=train_dataset,test_dataset = test_dataset,n_experiences=1,task_labels=False,seed=1234)

In [8]:
x_train[0].dim(),y_train.dim()

(1, 1)

# Attempt SimpleMLP

In [9]:
model_2=SimpleMLP(num_classes=6,input_size=70)

In [10]:
text_logger = TextLogger(open('./logs/Cumulative_fiveclass.txt', 'a'))
interactive_logger = InteractiveLogger()
csv_logger=CSVLogger("./logs/Cumulative_fiveclass.csv")


eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=NC_benchmark.n_classes, save_image=False,
                             stream=True),
    loggers=[interactive_logger, text_logger,csv_logger]
)
# Strategy
cl_strategy = Cumulative(
    model_2, Adam(model_2.parameters(), lr=0.001),
    CrossEntropyLoss(), train_mb_size=256, train_epochs=1, eval_mb_size=300,
    evaluator=eval_plugin)

In [11]:
y_train[0].size()

torch.Size([])

In [12]:

print('Starting experiment...')
results = []
for experience in NC_benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(NC_benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0]
-- >> Start of training phase << --
100%|██████████| 7104/7104 [00:47<00:00, 149.19it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0060
	Loss_MB/train_phase/train_stream/Task000 = 0.0000
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 47.6312
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9996
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 1516/1516 [00:10<00:00, 141.38it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 1.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████|

In [13]:
torch.__version__

'1.13.1'